In [1]:
# imports needed and logging
import gensim, pandas as pd, re
import numpy as np

event_data = pd.read_csv('event_data4.csv')
descriptions = []

for description in event_data[' description']:
    description = re.sub("[^a-zA-Z]", " ", description)
    description = description.lower().split()
    descriptions += [description]

/Users/rvenguswamy/anaconda3/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
model = gensim.models.Word2Vec(descriptions, size=100, window=10, min_count=2, workers=10)  
model.train(descriptions, total_examples=len(descriptions), epochs=10)

(17464338, 21652400)

In [3]:
model.wv.most_similar(positive=['dance'])

[('dancers', 0.5898144841194153),
 ('djs', 0.5778520107269287),
 ('performers', 0.5598466396331787),
 ('dancing', 0.5462778210639954),
 ('disco', 0.5376554727554321),
 ('hop', 0.5258179903030396),
 ('mix', 0.5219536423683167),
 ('trefft', 0.5212621688842773),
 ('sing', 0.5173811912536621),
 ('rave', 0.5113421678543091)]

In [9]:
def find_sim_event(query, description):
    
    sims = model.wv.most_similar(positive = query)
    sims = list(zip(*sims))
    sim_words = list(sims[0])
    sim_words.append(query)
    
    
    description_total = 0

        ##finding ones that have similar words
    simcount = 0
    for sim_word in sim_words:
        simcount += description.count(sim_word)
    if len(description) > 0:
        description_score_sim = simcount/len(description)
    else:
        description_score_sim = 0
    
        #### adding similarity across all words
    for word in description:
        try:
            word_score = model.wv.similarity(w1=query, w2=word)
        except:
            word_score = 0

        description_total += word_score
    if len(description) > 0:
        description_score = description_total
    else:
        description_score = 0
    description_score = description_score*description_score_sim
    return description_score
    
        ##combining both
def comb_str_sim(lquery, top):
    
    event_scores = []
    
    for i in range(len(descriptions)):
        
        description = descriptions[i]
        score = 0
        
        for query in lquery:
            score += find_sim_event(query, description)
        score = score/len(lquery)
        
    
        event_scores.append((event_data['url'][i], score, event_data[' description'][i]))
    event_scores.sort(key=lambda event: event[1], reverse=True)
    return(event_scores[:top])
    

In [10]:
comb_str_sim(['party','drinks'], 6)

[('https://www.eventbrite.co.uk/e/magaluf-69-events-package-2019-tickets-24907589270?aff=ebapi', 3.6749726987303672, 'MAGALUF    EVENTS PACKAGE  Magaluf      s Best Value Events Package  The Very Best Magaluf Events Put Together For Your Partying Needs    MAGALUF SUNSET BOOZE CRUISE VIP  Sold separately in Magaluf for         The boat party takes place every Wednesday  Friday  Saturday throughout the summer  The Sunset Booze Cruise has featured on Geordie Shore  Magaluf Weekender and not to mention the Inbetweeners  Enjoy the unlimited free bar  live DJ s  swim stops and crazy games    MAGALUF FULL MOON PARTY  Sold separately in Magaluf for         The party takes place at the exclusive Ocean Beach Club every Sunday   Thursday throughout the summer expect UV body painters  live DJ s  fire breathers Go Go Dancers    MAGALUF WHITE PARTY  Sold separately in Magaluf for         Every Monday   Friday The White Party Magaluf is new for Summer       Magaluf White Party brings its class and is

In [11]:
alexaquery = 'basketball game'
stringquerylist = alexaquery.split()
comb_str_sim(stringquerylist)

['basketball', 'game']


In [ ]:
model.save("modelw2v")